In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## yolov-12

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

In [ ]:
classes = ["Glioma", "Meningioma", "No Tumor", "Pituitary"]

In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO('yolo12m.pt')  # Ensure this weight file exists

# Train the model
model.train(data='/kaggle/input/brain-tumor-dataset/data.yaml', epochs=30, batch=15)

In [ ]:
best_model = YOLO('/kaggle/working/runs/detect/train4/weights/best.pt')

In [ ]:
results = best_model(['/kaggle/input/brain-tumor-dataset/train/images/Tr-gl_0124_jpg.rf.167286373f035bde76f3cc25fc6eed91.jpg'])  # return a list of Results objects
result = results[0]

In [ ]:
plt.imshow(result.plot())

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load local image (no need for requests)
image_path = '/kaggle/input/brain-tumor-dataset/train/images/Tr-gl_0124_jpg.rf.167286373f035bde76f3cc25fc6eed91.jpg'
img = Image.open(image_path).convert("RGB")

# Predict using YOLO
result = best_model.predict(img)[0]

# Display result
plt.figure(figsize=(8, 8))
plt.imshow(img)
ax = plt.gca()

# Draw boxes
for detection in result.boxes:
    x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy()
    conf = detection.conf[0].cpu().numpy()
    cls = detection.cls[0].cpu().numpy()
    rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                         linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)
    plt.text(x1, y1, f"{classes[int(cls)]} {conf:.2f}", color='white',
             fontsize=12, backgroundcolor='red')

plt.title('YOLOv12 Detection on Sample Image')
plt.axis('off')
plt.show()


In [ ]:
import cv2
from ultralytics import YOLO
import os

# === STEP 1: Define paths ===
video_path = "/kaggle/input/mri-video/3396561117-preview.mp4"  # 🔁 Replace with your uploaded file
output_path = "/kaggle/working/annotated_output.mp4"
model_path = "/kaggle/working/runs/detect/train4/weights/best.pt"  # 🔁 Replace with your trained model path

# === STEP 2: Load YOLOv8 model ===
model = YOLO(model_path)

# === STEP 3: Open the uploaded video ===
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === STEP 4: Define video writer ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# === STEP 5: Predict and write bounding boxes ===
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, verbose=False)[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        label = f"{model.names[cls]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    out.write(frame)

cap.release()
out.release()

print("✅ Annotated video saved at:", output_path)


In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# === STEP 1: Define paths ===
video_path = "/kaggle/input/mri-video/3396561117-preview.mp4"
output_path = "/kaggle/working/annotated_output.mp4"
model_path = "/kaggle/working/runs/detect/train4/weights/best.pt"

# === STEP 2: Your custom class names ===
custom_classes = ["Glioma", "Meningioma", "No Tumor", "Pituitary"]

# === STEP 3: Load the YOLO model ===
model = YOLO(model_path)

# === STEP 4: Open video ===
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === STEP 5: Set up video writer ===
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# === STEP 6: Predict and annotate ===
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    results = model.predict(frame, verbose=False)[0]
    results.names = {i: name for i, name in enumerate(custom_classes)}  # ✅ Set names in result

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        label = f"{results.names[cls]} {conf:.2f}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

    # Show first 3 frames
    if frame_count <= 3:
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title(f"Frame {frame_count}")
        plt.axis('off')
        plt.show()

cap.release()
out.release()

print("✅ Annotated video saved at:", output_path)
